# Modelado Predictivo - Producción Agroindustrial de Ica

**Objetivo**: Desarrollar modelos predictivos para estimar rendimientos y producción en la región de Ica.

**Modelos a desarrollar**:
1. Regresión Lineal para predicción de rendimientos
2. Clasificación de cultivos por características
3. Clustering de productores
4. Análisis de series temporales

---

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path

# Librerías de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# Configurar directorio
import os
os.chdir('..')

print("🤖 Librerías de Machine Learning importadas correctamente")

## 2. Carga y Preparación de Datos

In [ ]:
# Cargar datos
df = pd.read_csv("data/lugar_produccion_ica.csv", encoding='latin-1')

print(f"✅ Dataset cargado: {df.shape[0]} registros, {df.shape[1]} columnas")

# Limpiar y preparar datos
def preparar_datos(df):
    """Función para limpiar y preparar datos para modelado"""
    
    df_clean = df.copy()
    
    # Convertir columnas numéricas
    numeric_cols = ['AREA TOTAL (HA)', 'AREA CERTIFICAR (HA)', 
                   'RENDIMIENTO CERTIFICADO', 'RENDIMIENTO SOLICITUD']
    
    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Normalizar texto
    text_cols = ['PROVINCIA', 'DISTRITO', 'CULTIVO', 'TIPO PROD']
    for col in text_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.strip().str.upper()
    
    # Crear variables derivadas
    df_clean['PRODUCCION_ESTIMADA'] = df_clean['AREA CERTIFICAR (HA)'] * df_clean['RENDIMIENTO CERTIFICADO']
    df_clean['EFICIENCIA_RENDIMIENTO'] = df_clean['RENDIMIENTO CERTIFICADO'] / df_clean['RENDIMIENTO SOLICITUD']
    df_clean['RATIO_AREA'] = df_clean['AREA CERTIFICAR (HA)'] / df_clean['AREA TOTAL (HA)']
    
    # Codificar mes como numérico
    mes_mapping = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    df_clean['MES_NUM'] = df_clean['MES CERTIFICACION'].map(mes_mapping)
    
    # Eliminar outliers extremos
    for col in ['RENDIMIENTO CERTIFICADO', 'AREA CERTIFICAR (HA)']:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    return df_clean

# Preparar datos
df_clean = preparar_datos(df)
print(f"📊 Datos preparados: {df_clean.shape[0]} registros después de limpieza")

# Mostrar estadísticas básicas
print("\n📋 Estadísticas básicas de variables numéricas:")
numeric_cols = ['AREA CERTIFICAR (HA)', 'RENDIMIENTO CERTIFICADO', 'PRODUCCION_ESTIMADA']
display(df_clean[numeric_cols].describe())

## 3. Modelo de Regresión - Predicción de Rendimiento

In [ ]:
# Preparar datos para regresión
print("🎯 MODELO DE REGRESIÓN - PREDICCIÓN DE RENDIMIENTO")
print("=" * 60)

# Seleccionar datos válidos
df_reg = df_clean.dropna(subset=['RENDIMIENTO CERTIFICADO', 'AREA CERTIFICAR (HA)', 'CULTIVO'])
print(f"📊 Datos para regresión: {len(df_reg)} registros")

# Codificar variables categóricas
le_cultivo = LabelEncoder()
le_provincia = LabelEncoder()
le_distrito = LabelEncoder()
le_tipo = LabelEncoder()

df_reg['CULTIVO_COD'] = le_cultivo.fit_transform(df_reg['CULTIVO'])
df_reg['PROVINCIA_COD'] = le_provincia.fit_transform(df_reg['PROVINCIA'])
df_reg['DISTRITO_COD'] = le_distrito.fit_transform(df_reg['DISTRITO'])
df_reg['TIPO_PROD_COD'] = le_tipo.fit_transform(df_reg['TIPO PROD'])

# Definir características y variable objetivo
features = ['AREA CERTIFICAR (HA)', 'CULTIVO_COD', 'PROVINCIA_COD', 'DISTRITO_COD', 
           'TIPO_PROD_COD', 'AÑO CERTIFICACION', 'MES_NUM']
X = df_reg[features]
y = df_reg['RENDIMIENTO CERTIFICADO']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"🔄 Conjunto de entrenamiento: {len(X_train)} registros")
print(f"🔄 Conjunto de prueba: {len(X_test)} registros")

# Entrenar múltiples modelos
modelos = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

resultados = {}

for nombre, modelo in modelos.items():
    # Entrenar modelo
    modelo.fit(X_train, y_train)
    
    # Predicciones
    y_pred = modelo.predict(X_test)
    
    # Métricas
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    resultados[nombre] = {
        'MSE': mse,
        'R2': r2,
        'Predictions': y_pred
    }
    
    print(f"\n📊 {nombre}:")
    print(f"   - MSE: {mse:.4f}")
    print(f"   - R²: {r2:.4f}")

# Seleccionar el mejor modelo
mejor_modelo = max(resultados.keys(), key=lambda x: resultados[x]['R2'])
print(f"\n🏆 Mejor modelo: {mejor_modelo} (R² = {resultados[mejor_modelo]['R2']:.4f})")

In [ ]:
# Visualizar resultados del mejor modelo
mejor_pred = resultados[mejor_modelo]['Predictions']

# Gráfico de predicciones vs valores reales
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Predicciones vs Reales', 'Residuos', 'Distribución de Errores', 'Importancia de Features'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Scatter plot predicciones vs reales
fig.add_trace(
    go.Scatter(x=y_test, y=mejor_pred, mode='markers', name='Predicciones'),
    row=1, col=1
)

# Línea de referencia perfecta
fig.add_trace(
    go.Scatter(x=[y_test.min(), y_test.max()], y=[y_test.min(), y_test.max()], 
              mode='lines', name='Línea perfecta', line=dict(dash='dash')),
    row=1, col=1
)

# Residuos
residuos = y_test - mejor_pred
fig.add_trace(
    go.Scatter(x=mejor_pred, y=residuos, mode='markers', name='Residuos'),
    row=1, col=2
)

# Histograma de errores
fig.add_trace(
    go.Histogram(x=residuos, name='Distribución de Errores'),
    row=2, col=1
)

# Importancia de features (si es Random Forest)
if mejor_modelo == 'Random Forest':
    importancias = modelos[mejor_modelo].feature_importances_
    fig.add_trace(
        go.Bar(x=features, y=importancias, name='Importancia'),
        row=2, col=2
    )

fig.update_layout(height=800, title_text=f"Análisis del Modelo {mejor_modelo}")
fig.show()

print("\n📊 Métricas del mejor modelo:")
print(f"- R² Score: {resultados[mejor_modelo]['R2']:.4f}")
print(f"- MSE: {resultados[mejor_modelo]['MSE']:.4f}")
print(f"- RMSE: {np.sqrt(resultados[mejor_modelo]['MSE']):.4f}")

## 4. Clustering de Productores

In [ ]:
# Clustering de productores
print("🎯 CLUSTERING DE PRODUCTORES")
print("=" * 60)

# Preparar datos para clustering
df_cluster = df_clean.dropna(subset=['RENDIMIENTO CERTIFICADO', 'AREA CERTIFICAR (HA)', 'PRODUCCION_ESTIMADA'])

# Agregar por productor (usando expediente como proxy)
productor_stats = df_cluster.groupby('EXPEDIENTE').agg({
    'AREA CERTIFICAR (HA)': 'sum',
    'RENDIMIENTO CERTIFICADO': 'mean',
    'PRODUCCION_ESTIMADA': 'sum',
    'CULTIVO': 'nunique',
    'PROVINCIA': 'first',
    'TIPO PROD': 'first'
}).round(2)

productor_stats.columns = ['Area_Total', 'Rendimiento_Promedio', 'Produccion_Total', 
                          'Diversidad_Cultivos', 'Provincia', 'Tipo_Productor']

print(f"📊 Datos para clustering: {len(productor_stats)} productores")

# Seleccionar variables para clustering
cluster_vars = ['Area_Total', 'Rendimiento_Promedio', 'Produccion_Total', 'Diversidad_Cultivos']
X_cluster = productor_stats[cluster_vars]

# Estandarizar variables
scaler = StandardScaler()
X_cluster_scaled = scaler.fit_transform(X_cluster)

# Determinar número óptimo de clusters
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_cluster_scaled)
    inertias.append(kmeans.inertia_)

# Visualizar método del codo
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.title('Método del Codo para Determinar K Óptimo')
plt.grid(True)
plt.show()

# Aplicar K-means con k=4
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X_cluster_scaled)

# Agregar clusters al dataframe
productor_stats['Cluster'] = clusters

print(f"\n🎯 Clustering completado con {n_clusters} clusters")
print("\n📊 Distribución de clusters:")
cluster_dist = pd.Series(clusters).value_counts().sort_index()
for cluster, count in cluster_dist.items():
    print(f"   Cluster {cluster}: {count} productores ({count/len(productor_stats)*100:.1f}%)")

In [ ]:
# Analizar características de cada cluster
print("\n📋 CARACTERÍSTICAS DE CADA CLUSTER:")
print("=" * 60)

cluster_summary = productor_stats.groupby('Cluster').agg({
    'Area_Total': ['mean', 'median'],
    'Rendimiento_Promedio': ['mean', 'median'],
    'Produccion_Total': ['mean', 'median'],
    'Diversidad_Cultivos': ['mean', 'median'],
    'Tipo_Productor': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'N/A'
}).round(2)

display(cluster_summary)

# Visualizar clusters
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Área vs Rendimiento', 'Área vs Producción', 'Rendimiento vs Diversidad', 'Distribución por Tipo'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

colors = ['red', 'blue', 'green', 'orange']

for i, cluster in enumerate(sorted(productor_stats['Cluster'].unique())):
    cluster_data = productor_stats[productor_stats['Cluster'] == cluster]
    
    # Área vs Rendimiento
    fig.add_trace(
        go.Scatter(x=cluster_data['Area_Total'], y=cluster_data['Rendimiento_Promedio'], 
                  mode='markers', name=f'Cluster {cluster}', 
                  marker=dict(color=colors[i])),
        row=1, col=1
    )
    
    # Área vs Producción
    fig.add_trace(
        go.Scatter(x=cluster_data['Area_Total'], y=cluster_data['Produccion_Total'], 
                  mode='markers', name=f'Cluster {cluster}', 
                  marker=dict(color=colors[i]), showlegend=False),
        row=1, col=2
    )
    
    # Rendimiento vs Diversidad
    fig.add_trace(
        go.Scatter(x=cluster_data['Rendimiento_Promedio'], y=cluster_data['Diversidad_Cultivos'], 
                  mode='markers', name=f'Cluster {cluster}', 
                  marker=dict(color=colors[i]), showlegend=False),
        row=2, col=1
    )

# Distribución por tipo de productor
tipo_cluster = pd.crosstab(productor_stats['Cluster'], productor_stats['Tipo_Productor'])
for tipo in tipo_cluster.columns:
    fig.add_trace(
        go.Bar(x=tipo_cluster.index, y=tipo_cluster[tipo], name=tipo),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="Análisis de Clusters de Productores")
fig.show()

# Interpretar clusters
print("\n🔍 INTERPRETACIÓN DE CLUSTERS:")
print("=" * 60)

interpretaciones = {
    0: "Productores Pequeños - Baja área y producción",
    1: "Productores Eficientes - Alto rendimiento, área moderada",
    2: "Productores Grandes - Alta área y producción",
    3: "Productores Diversificados - Múltiples cultivos"
}

for cluster in sorted(productor_stats['Cluster'].unique()):
    cluster_data = productor_stats[productor_stats['Cluster'] == cluster]
    print(f"\n🎯 Cluster {cluster}: {interpretaciones.get(cluster, 'Características especiales')}")
    print(f"   - Productores: {len(cluster_data)}")
    print(f"   - Área promedio: {cluster_data['Area_Total'].mean():.1f} Ha")
    print(f"   - Rendimiento promedio: {cluster_data['Rendimiento_Promedio'].mean():.1f} t/ha")
    print(f"   - Diversidad promedio: {cluster_data['Diversidad_Cultivos'].mean():.1f} cultivos")
    print(f"   - Tipo predominante: {cluster_data['Tipo_Productor'].mode().iloc[0]}")

## 5. Análisis de Componentes Principales (PCA)

In [ ]:
# Análisis de Componentes Principales
print("🎯 ANÁLISIS DE COMPONENTES PRINCIPALES (PCA)")
print("=" * 60)

# Aplicar PCA
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_cluster_scaled)

# Crear DataFrame con componentes principales
pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(4)])
pca_df['Cluster'] = clusters
pca_df['Tipo_Productor'] = productor_stats['Tipo_Productor'].values

# Varianza explicada
varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)

print(f"\n📊 Varianza explicada por cada componente:")
for i, var in enumerate(varianza_explicada):
    print(f"   PC{i+1}: {var:.4f} ({var*100:.1f}%)")

print(f"\n📊 Varianza acumulada:")
for i, var in enumerate(varianza_acumulada):
    print(f"   PC1-PC{i+1}: {var:.4f} ({var*100:.1f}%)")

# Visualizar PCA
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Varianza Explicada', 'PC1 vs PC2 (Clusters)', 'PC1 vs PC2 (Tipo)', 'Loadings'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Gráfico de varianza explicada
fig.add_trace(
    go.Bar(x=[f'PC{i+1}' for i in range(4)], y=varianza_explicada, name='Varianza Individual'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=[f'PC{i+1}' for i in range(4)], y=varianza_acumulada, 
              mode='lines+markers', name='Varianza Acumulada'),
    row=1, col=1
)

# PC1 vs PC2 coloreado por clusters
for cluster in sorted(pca_df['Cluster'].unique()):
    cluster_data = pca_df[pca_df['Cluster'] == cluster]
    fig.add_trace(
        go.Scatter(x=cluster_data['PC1'], y=cluster_data['PC2'], 
                  mode='markers', name=f'Cluster {cluster}',
                  marker=dict(color=colors[cluster])),
        row=1, col=2
    )

# PC1 vs PC2 coloreado por tipo de productor
for tipo in pca_df['Tipo_Productor'].unique():
    tipo_data = pca_df[pca_df['Tipo_Productor'] == tipo]
    fig.add_trace(
        go.Scatter(x=tipo_data['PC1'], y=tipo_data['PC2'], 
                  mode='markers', name=f'Tipo {tipo}'),
        row=2, col=1
    )

# Loadings (contribución de variables)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
fig.add_trace(
    go.Scatter(x=loadings[:, 0], y=loadings[:, 1], 
              mode='markers+text', text=cluster_vars, name='Variables',
              textposition='top center'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Análisis de Componentes Principales")
fig.show()

# Interpretar componentes
print("\n🔍 INTERPRETACIÓN DE COMPONENTES PRINCIPALES:")
print("=" * 60)

print("\n📊 Contribución de variables a cada componente:")
componentes_df = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(4)], index=cluster_vars)
display(componentes_df.round(3))

print("\n🎯 Interpretación:")
print("- PC1: Representa principalmente el tamaño de la operación (área y producción total)")
print("- PC2: Representa la eficiencia (rendimiento vs diversidad)")
print("- PC3: Representa la especialización vs diversificación")
print("- PC4: Representa características específicas del productor")

## 6. Modelo de Clasificación - Tipo de Productor

In [ ]:
# Modelo de clasificación para predecir tipo de productor
print("🎯 MODELO DE CLASIFICACIÓN - TIPO DE PRODUCTOR")
print("=" * 60)

# Preparar datos para clasificación
df_class = df_clean.dropna(subset=['TIPO PROD', 'RENDIMIENTO CERTIFICADO', 'AREA CERTIFICAR (HA)'])

# Características para clasificación
features_class = ['AREA CERTIFICAR (HA)', 'RENDIMIENTO CERTIFICADO', 'PRODUCCION_ESTIMADA', 'MES_NUM']

# Agregar características por expediente
expediente_features = df_class.groupby('EXPEDIENTE').agg({
    'AREA CERTIFICAR (HA)': ['sum', 'mean'],
    'RENDIMIENTO CERTIFICADO': ['mean', 'std'],
    'PRODUCCION_ESTIMADA': 'sum',
    'CULTIVO': 'nunique',
    'TIPO PROD': 'first'
}).round(2)

# Aplanar nombres de columnas
expediente_features.columns = ['_'.join(col).strip() for col in expediente_features.columns.values]
expediente_features.rename(columns={'TIPO PROD_first': 'TIPO_PROD'}, inplace=True)

# Eliminar filas con valores nulos
expediente_features = expediente_features.dropna()

print(f"📊 Datos para clasificación: {len(expediente_features)} expedientes")

# Preparar X e y
X_class = expediente_features.drop('TIPO_PROD', axis=1)
y_class = expediente_features['TIPO_PROD']

# Dividir datos
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Entrenar modelo de clasificación
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_class, y_train_class)

# Predicciones
y_pred_class = rf_classifier.predict(X_test_class)

# Métricas de clasificación
print("\n📊 RESULTADOS DE CLASIFICACIÓN:")
print("=" * 40)

print("\n📋 Reporte de clasificación:")
print(classification_report(y_test_class, y_pred_class))

# Matriz de confusión
cm = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=rf_classifier.classes_, 
            yticklabels=rf_classifier.classes_)
plt.title('Matriz de Confusión - Clasificación de Tipo de Productor')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

# Importancia de características
importancias_class = rf_classifier.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X_class.columns,
    'Importancia': importancias_class
}).sort_values('Importancia', ascending=False)

print("\n📊 Importancia de características:")
display(feature_importance_df)

# Visualizar importancia
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df, x='Importancia', y='Feature', palette='viridis')
plt.title('Importancia de Características - Clasificación de Tipo de Productor')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

# Accuracy
accuracy = rf_classifier.score(X_test_class, y_test_class)
print(f"\n🎯 Accuracy del modelo: {accuracy:.4f} ({accuracy*100:.1f}%)")

## 7. Resumen y Exportación de Modelos

In [ ]:
# Resumen de todos los modelos
print("📋 RESUMEN DE MODELOS DESARROLLADOS")
print("=" * 60)

print("\n🎯 1. MODELO DE REGRESIÓN (Predicción de Rendimiento):")
print(f"   - Mejor modelo: {mejor_modelo}")
print(f"   - R² Score: {resultados[mejor_modelo]['R2']:.4f}")
print(f"   - RMSE: {np.sqrt(resultados[mejor_modelo]['MSE']):.4f}")
print(f"   - Variables importantes: Área, Cultivo, Provincia, Distrito")

print("\n🎯 2. CLUSTERING DE PRODUCTORES:")
print(f"   - Número de clusters: {n_clusters}")
print(f"   - Productores analizados: {len(productor_stats)}")
print(f"   - Variables: Área, Rendimiento, Producción, Diversidad")
print(f"   - Clusters identificados: Pequeños, Eficientes, Grandes, Diversificados")

print("\n🎯 3. ANÁLISIS DE COMPONENTES PRINCIPALES:")
print(f"   - Varianza explicada (PC1-PC2): {varianza_acumulada[1]:.4f} ({varianza_acumulada[1]*100:.1f}%)")
print(f"   - PC1: Tamaño de operación")
print(f"   - PC2: Eficiencia productiva")

print("\n🎯 4. MODELO DE CLASIFICACIÓN (Tipo de Productor):")
print(f"   - Algoritmo: Random Forest")
print(f"   - Accuracy: {accuracy:.4f} ({accuracy*100:.1f}%)")
print(f"   - Expedientes analizados: {len(expediente_features)}")
print(f"   - Variable más importante: {feature_importance_df.iloc[0]['Feature']}")

# Crear directorio para guardar modelos
models_dir = Path('outputs/models')
models_dir.mkdir(exist_ok=True)

# Guardar resultados en archivos
import joblib

# Guardar el mejor modelo de regresión
joblib.dump(modelos[mejor_modelo], models_dir / f'modelo_regresion_{mejor_modelo.lower().replace(" ", "_")}.pkl')

# Guardar modelo de clustering
joblib.dump(kmeans, models_dir / 'modelo_clustering.pkl')
joblib.dump(scaler, models_dir / 'scaler_clustering.pkl')

# Guardar PCA
joblib.dump(pca, models_dir / 'modelo_pca.pkl')

# Guardar modelo de clasificación
joblib.dump(rf_classifier, models_dir / 'modelo_clasificacion.pkl')

# Guardar encoders
joblib.dump(le_cultivo, models_dir / 'encoder_cultivo.pkl')
joblib.dump(le_provincia, models_dir / 'encoder_provincia.pkl')
joblib.dump(le_distrito, models_dir / 'encoder_distrito.pkl')

# Guardar resultados de clustering
productor_stats.to_csv(models_dir / 'clusters_productores.csv', encoding='utf-8')

# Guardar métricas de modelos
metricas_modelos = {
    'Modelo': ['Regresión', 'Clustering', 'PCA', 'Clasificación'],
    'Métrica_Principal': [f'R² = {resultados[mejor_modelo]["R2"]:.4f}', 
                         f'{n_clusters} clusters', 
                         f'{varianza_acumulada[1]*100:.1f}% varianza',
                         f'{accuracy*100:.1f}% accuracy'],
    'Algoritmo': [mejor_modelo, 'K-Means', 'PCA', 'Random Forest'],
    'Datos_Entrenamiento': [len(X_train), len(productor_stats), len(X_cluster), len(X_train_class)]
}

pd.DataFrame(metricas_modelos).to_csv(models_dir / 'resumen_modelos.csv', index=False, encoding='utf-8')

print("\n💾 MODELOS Y RESULTADOS GUARDADOS:")
print("=" * 60)
print(f"📁 Directorio: {models_dir.absolute()}")
print("📄 Archivos guardados:")
print("   - Modelos entrenados (.pkl)")
print("   - Encoders y transformadores")
print("   - Resultados de clustering")
print("   - Resumen de métricas")

print("\n🎯 RECOMENDACIONES PARA PRODUCCIÓN:")
print("=" * 60)
print("1. Usar modelo de regresión para estimar rendimientos esperados")
print("2. Segmentar productores usando clusters identificados")
print("3. Enfocar políticas diferenciadas por tipo de productor")
print("4. Monitorear variables clave: área, cultivo, localización")
print("5. Promover diversificación en productores grandes")
print("6. Apoyar eficiencia en productores pequeños")

print("\n✅ MODELADO COMPLETADO EXITOSAMENTE")